In [1]:
from environment import Environment
import numpy as np
from game import SumUnanimityGames, GlobalFeatureImportance, LocalFeatureImportance, UnsupervisedFeatureImportance
from plot import plot
import pandas as pd

In [2]:
from algorithms.CMCS import *
from algorithms.GapE import GapE
from algorithms.SAR import SAR
from algorithms.ApproShapley import ApproShapley
from algorithms.BUS import BUS
from algorithms.HybridApproBUS import SmartHybridApproBUS
from algorithms.SVARM import SVARM, StratSVARM
from algorithms.ShapleySort import ShapleySort
from algorithms.KernelSHAP import KernelSHAP
from algorithms.shap_k import SHAP_K
%load_ext autoreload
%autoreload 2
%matplotlib inline

c:\Users\Public\anaconda3\envs\shapley\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
df = pd.read_csv("results/data/PAC/pac_k=5_tmin=30_delta=0.01_epsilon=0.0005_temp.csv", index_col=0)
df[["SamplingSHAP@K", "CMCS", "CMCS@K", "Greedy CMCS"]] = df[["SamplingSHAP@K", "CMCS", "CMCS@K", "Greedy CMCS"]].astype(int)
df.to_csv(f"results/data/PAC/pac_k=5_tmin=30_delta=0.01_epsilon=0.0005_round.csv")

In [4]:
k=5
rounds=200
metric = "ratio"
env = Environment(budget=-1, metric=metric)
results = []

In [5]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

In [6]:
global_games = [("Diabetes regression random forest",10), ("Titanic classification random forest",11), ("Wine classification random forest",13)] #("Adult classification random forest", 14), ("German credit classification random forest", 20), ("Bike sharing regression random forest", 15), ("Bank marketing classification random forest", 16) ]
local_games = ["adult_classification"] #"nlp_sentiment", "image classifier", ]
unsupervised_games = [("vf_Fifa", 12)] #("vf_BigFive", 12), ("vf_BreastCancer", 9)
global_games = [GlobalFeatureImportance(filepath=f"datasets/Global feature importance/{filepath}.csv", num_players=n) for (filepath,n) in global_games]
local_games = [LocalFeatureImportance(directory=f"datasets/{folder}", num_players=14) for folder in local_games]
unsupervised_games = [UnsupervisedFeatureImportance(filepath=f"datasets/Unsupervised Feature Importance/{filepath}.csv", num_players=n) for (filepath,n) in unsupervised_games]
games = global_games #unsupervised_games #global_games + local_games + 

[0.000 40.776 -31.962 ... 1498.452 1512.220 1523.923]
[56.782 28.189 638.677 73.494 -125.712 -126.210 150.160 179.125 479.528
 169.890] 1523.9230539837763
could not find cached values. manual reindexing...
1000
2000
could not find cached shapley values. manual calculation...
[0.000 -0.011 0.108 ... 0.220 0.228 0.201]
[-0.006 0.024 -0.002 0.116 0.002 0.004 0.005 0.022 0.027 0.011 -0.002] 0.2014925373134333
[0.000 0.204 0.222 ... 0.593 0.593 0.574]
[0.066 0.043 0.012 0.026 0.031 0.036 0.080 0.015 0.006 0.079 0.050 0.046
 0.086] 0.5740740740740702
[0.000 0.000 0.000 ... 34.350 34.350 37.935]
[3.275 3.275 3.123 3.275 3.123 3.275 3.123 3.123 3.275 2.972 2.972 3.123] 37.934587507932406


In [7]:
t_min = 30
delta = 0.01
epsilon = 0.0005

In [10]:
for game in games:
    results += [(game.name, f"SamplingSHAP@K", env.evaluate_PAC(game, SHAP_K(t_min, delta, epsilon), k, epsilon, rounds))]
    results += [(game.name, f"CMCS", env.evaluate_PAC(game, CMCS(t_min, delta, epsilon), k, epsilon, rounds))]
    results += [(game.name, f"CMCS@K", env.evaluate_PAC(game, CMCS_at_K(t_min, delta, epsilon), k, epsilon, rounds))]
    results += [(game.name, f"Greedy CMCS", env.evaluate_PAC(game, Greedy_CMCS(t_min, delta, epsilon), k, epsilon, rounds))]
    

100%|██████████| 200/200 [00:18<00:00, 10.86it/s, accuracy=1.0, func_calls=2617, topk_approx=[2 6 7 8 9], topk_real=[2 6 7 8 9]]


3723.79 94.1345267429843 1.0


100%|██████████| 200/200 [00:31<00:00,  6.37it/s, accuracy=1.0, func_calls=7744, topk_approx=[2 6 7 8 9], topk_real=[2 6 7 8 9]] 


7793.9 143.69041802136906 1.0


100%|██████████| 200/200 [00:17<00:00, 11.20it/s, accuracy=1.0, func_calls=2377, topk_approx=[2 6 7 8 9], topk_real=[2 6 7 8 9]]


...

120859.6 1906.7297825645335 1.0


100%|██████████| 200/200 [02:55<00:00,  1.14it/s, accuracy=0.995, func_calls=31131, topk_approx=[ 0  6  9 10 12], topk_real=[ 0  6  9 10 12]]             


29913.685 641.4122550148941 0.995


100%|██████████| 200/200 [07:45<00:00,  2.33s/it, accuracy=1.0, func_calls=34315, topk_approx=[ 0  6  9 10 12], topk_real=[ 0  6  9 10 12]]

34265.77 501.77920662028475 1.0


In [11]:
results

[('Diabetes regression random forest',
  'SamplingSHAP@K',
  (3723.79, 94.1345267429843, 1.0)),
 ('Diabetes regression random forest',
  'CMCS',
  (7793.9, 143.69041802136906, 1.0)),
 ('Diabetes regression random forest',
  'CMCS@K',
  (2976.81, 55.70041472469045, 1.0)),
 ('Diabetes regression random forest',
  'Greedy CMCS',
  (4593.885, 85.8339990085748, 1.0)),
 ('Titanic classification random forest',
  'SamplingSHAP@K',
  (4852.385, 113.95817593832815, 1.0)),
 ('Titanic classification random forest',
  'CMCS',
  (11036.54, 237.34245161515773, 1.0)),
 ('Titanic classification random forest',
  'CMCS@K',
  (3884.355, 72.8894781501418, 1.0)),
 ('Titanic classification random forest',
  'Greedy CMCS',
  (5782.5, 124.55654141814573, 1.0)),
 ('Wine classification random forest',
  'SamplingSHAP@K',
  (34953.235, 1046.3809084298641, 0.995)),
 ('Wine classification random forest',
  'CMCS',
  (120859.6, 1906.7297825645335, 1.0)),
 ('Wine classification random forest',
  'CMCS@K',
  (29913.

In [12]:
df_dict = {}
for game, method, (value, se,acc) in results:
    if game not in df_dict:
        df_dict[game] = {}
        df_dict[game + " - SE"] = {}
    df_dict[game][method] = value
    df_dict[game + " - SE"][method] = se

# Create DataFrame
df = pd.DataFrame.from_dict(df_dict, orient='index')
df

,SamplingSHAP@K,CMCS,CMCS@K,Greedy CMCS
Diabetes regression random forest,3723.790000,7793.900000,2976.810000,4593.885000
Diabetes regression random forest - SE,94.134527,143.690418,55.700415,85.833999
Titanic classification random forest,4852.385000,11036.540000,3884.355000,5782.500000
Titanic classification random forest - SE,113.958176,237.342452,72.889478,124.556541
Wine classification random forest,34953.235000,120859.600000,29913.685000,34265.770000
Wine classification random forest - SE,1046.380908,1906.729783,641.412255,501.779207


In [13]:
df.to_csv(f"results/data/PAC/pac_k={k}_tmin={t_min}_delta={delta}_epsilon={epsilon}.csv")